In [ ]:
from ramanchada2.io.HSDS import visit_domain, filter_dataset, read_cha
import h5pyd
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

In [ ]:
from ramanchada2.spectrum import from_chada
import math


def load_dataset(parentdomain,domain,results={}):
    with h5pyd.File(domain) as f:
        wavelength = f["annotation_study"].attrs["wavelength"]
        sample = f["annotation_sample"].attrs["sample"]
    spe = from_chada(domain,h5module=h5pyd)
    spe = spe - spe.moving_minimum(16)
    spe = spe.normalize()
    results[domain] = (spe.x,spe.y,wavelength,sample)

query_sample = "PST"

#visit_domain(topdomain="/Round_Robin_1/",process_dataset=print_dataset) 
results = {}
visit_domain("/Round_Robin_1/",
        process_dataset=filter_dataset,kwargs={"process_file" : load_dataset,"sample": query_sample, 
                            "kwargs" : {"results" : results}})

In [ ]:
def ramanshift2wavelength(shift,laser):
    return 1/((1/laser)-shift/10**7)
def wavelength2ramanshift(wavelength,laser):
    return 10**7/laser-10**7/wavelength   

domain_len = 1024+2048
print(domain_len)
xnew0 = np.arange(0,domain_len,1)
for lex in (532,633,785):
    xw = ramanshift2wavelength(xnew0,lex)
    #print(xw,min(xw),max(xw),len(xw))

for laser in (532,633,785):
    max_vw = ramanshift2wavelength(4000,laser)
    xw =  np.arange(laser,math.ceil(max_vw),(max_vw-laser)/domain_len)
    print("nm",laser,max_vw,len(xw),(max_vw-laser)/domain_len)
    x = wavelength2ramanshift(xw,laser)
    ax = plt.plot(xw,x,marker='o',label=laser)
    plt.ylabel('Raman shift, cm-1')
    plt.xlabel('wavelength, nm')
plt.legend()
plt.show()

In [ ]:
from matplotlib.pyplot import figure

plt.figure(figsize=(1,1))


def spectrum_interpolate(x,y, newx):
    f = interp1d(x, y)
    xmin = np.where(newx>=min(x))
    xmax = np.where(newx<=max(x))    
    #returns a tuple , if empty, then max(spe.x) is beyond the new x
    xmax = xmax[0][len(xmax[0])-1] if len(xmax[0]>0) else len(newx)
    xmin = xmin[0][0] if len(xmin[0]>0) else 0
    newy = np.zeros(len(newx))
    newy[xmin:(xmax+1)] = f(newx[xmin:(xmax+1)])        
    return newy

domain_len=2048
offset = 100


ff, (ax2) = plt.subplots(1, 1, sharey=True)
#just for fun if we want more than one vector
_x0=np.linspace(offset,2000,domain_len)
_x1=np.linspace(2000,4000,domain_len)
for domain in results:
    x,y,wavelength,sample = results[domain]
    f = interp1d(x, y)
    wavelength = int(wavelength)
    _y0 = spectrum_interpolate(x,y,_x0)
    _y1 = spectrum_interpolate(x,y,_x1)
    plt.plot(x,y, 'x')
    plt.plot(_x0,_y0, '-')
    plt.plot(_x1,_y1, 'x')
    plt.suptitle(domain)
    plt.show()